A tutorial on logistic models (for predicting risk of software changes)
======
Full data is at:
  http://mockus.org/riskTut.zip and

* Version 3.2.1b
    * Except where otherwise noted, the content on this site is licensed under a Creative Commons Attribution 3.0 License.
 http://creativecommons.org/licenses/by/3.0/

In [1]:
# For nicer printing
options(digits=2);

In [3]:
# Read in the data
risk <- read.table("risk1.paper",header=T);

#It typiaclly is a no-brainer for operational data: log-transform ALL the numeric variables
vars <- cbind (risk$isBad, log(risk$NS),log(risk$NM),log(risk$NF),log(risk$NLOGIN),
           log(risk$NMR),log(risk$ND),log(risk$LA+1),log(risk$LD+1),
           log(risk$LinesUnchanged+1),log(risk$to-risk$from+1),risk$logEXP,risk$logREXP,
           risk$logSEXP,risk$FIX);
dimnames(vars)[[2]] <- c("isBad","lNS", "lNM", "lNF", "lNLgn",  "lNMR", "lND",
                         "lLA", "lLD", "lLOC", "lINT", "lEXP", "lREXP", "lSEXP","FIX"); 

#R likes data.frame (as pythoin numpy): a matrix with (potentially) different types in each column
data <- data.frame(vars);

In [4]:
#now explore variables
summary(data);

     isBad           lNS            lNM           lNF          lNLgn    
 Min.   :0.00   Min.   :0.00   Min.   :0.0   Min.   :0.0   Min.   :0.0  
 1st Qu.:0.00   1st Qu.:0.00   1st Qu.:0.0   1st Qu.:0.0   1st Qu.:0.0  
 Median :0.00   Median :0.00   Median :0.0   Median :0.7   Median :0.0  
 Mean   :0.02   Mean   :0.29   Mean   :0.6   Mean   :1.0   Mean   :0.3  
 3rd Qu.:0.00   3rd Qu.:0.69   3rd Qu.:1.1   3rd Qu.:1.6   3rd Qu.:0.7  
 Max.   :1.00   Max.   :2.89   Max.   :6.5   Max.   :7.7   Max.   :3.6  
      lNMR          lND           lLA            lLD            lLOC     
 Min.   :0.0   Min.   :0.0   Min.   : 0.0   Min.   : 0.0   Min.   : 0.0  
 1st Qu.:0.0   1st Qu.:0.7   1st Qu.: 2.6   1st Qu.: 0.7   1st Qu.: 7.6  
 Median :0.7   Median :1.6   Median : 3.9   Median : 2.3   Median : 8.9  
 Mean   :0.9   Mean   :1.8   Mean   : 4.1   Mean   : 2.7   Mean   : 9.0  
 3rd Qu.:1.4   3rd Qu.:2.7   3rd Qu.: 5.4   3rd Qu.: 4.1   3rd Qu.:10.3  
 Max.   :5.7   Max.   :7.9   Max.   :14.4   M

### Interpret  basic summaries

In [5]:
cor(vars);# OK for normal distrubution
cor(vars,method="spearman"); #OK for any: uses ranks

,isBad,lNS,lNM,lNF,lNLgn,lNMR,lND,lLA,lLD,lLOC,lINT,lEXP,lREXP,lSEXP,FIX
isBad,1.0000,0.1283,0.1368,0.1308,0.1238,0.1420,0.1355,0.1280,0.1211,0.1179,0.0992,-0.0333,-0.0318,-0.0739,-0.0029
lNS,0.128,1.000,0.824,0.736,0.756,0.772,0.674,0.584,0.554,0.548,0.457,-0.022,-0.020,-0.616,-0.179
lNM,0.137,0.824,1.000,0.903,0.757,0.827,0.817,0.710,0.663,0.649,0.533,0.023,0.034,-0.445,-0.210
lNF,0.1308,0.7360,0.9027,1.0000,0.7491,0.8504,0.9078,0.8024,0.7435,0.6882,0.5735,0.0046,0.0174,-0.3916,-0.2463
lNLgn,0.124,0.756,0.757,0.749,1.000,0.808,0.733,0.651,0.634,0.601,0.592,-0.080,-0.082,-0.493,-0.231
lNMR,0.142,0.772,0.827,0.850,0.808,1.000,0.877,0.778,0.751,0.720,0.644,-0.053,-0.050,-0.455,-0.240
lND,0.136,0.674,0.817,0.908,0.733,0.877,1.000,0.886,0.856,0.792,0.635,-0.031,-0.014,-0.359,-0.227
lLA,0.1280,0.5836,0.7103,0.8024,0.6505,0.7776,0.8863,1.0000,0.8269,0.7243,0.5760,-0.0123,-0.0035,-0.3095,-0.1999
lLD,0.1211,0.5536,0.6630,0.7435,0.6338,0.7506,0.8557,0.8269,1.0000,0.6976,0.5616,-0.0200,-0.0027,-0.2836,-0.2210
lLOC,0.118,0.548,0.649,0.688,0.601,0.720,0.792,0.724,0.698,1.000,0.517,-0.043,-0.035,-0.275,-0.151


,isBad,lNS,lNM,lNF,lNLgn,lNMR,lND,lLA,lLD,lLOC,lINT,lEXP,lREXP,lSEXP,FIX
isBad,1.0000,0.1004,0.1084,0.1097,0.0999,0.1130,0.1128,0.1146,0.1052,0.1057,0.1039,-0.0374,-0.0336,-0.0661,-0.0029
lNS,0.100,1.000,0.755,0.650,0.644,0.652,0.577,0.512,0.477,0.489,0.436,-0.027,-0.019,-0.595,-0.149
lNM,0.1084,0.7551,1.0000,0.8561,0.6457,0.7432,0.7438,0.6532,0.5907,0.6050,0.5100,-0.0092,0.0063,-0.3879,-0.1735
lNF,0.1097,0.6501,0.8561,1.0000,0.6352,0.7702,0.8615,0.7571,0.6716,0.6472,0.5443,-0.0160,0.0031,-0.3268,-0.2093
lNLgn,0.100,0.644,0.646,0.635,1.000,0.696,0.638,0.577,0.558,0.551,0.620,-0.097,-0.091,-0.447,-0.213
lNMR,0.113,0.652,0.743,0.770,0.696,1.000,0.835,0.736,0.696,0.690,0.639,-0.059,-0.048,-0.382,-0.202
lND,0.113,0.577,0.744,0.862,0.638,0.835,1.000,0.868,0.819,0.772,0.609,-0.054,-0.031,-0.303,-0.193
lLA,0.115,0.512,0.653,0.757,0.577,0.736,0.868,1.000,0.776,0.707,0.558,-0.031,-0.018,-0.262,-0.172
lLD,0.105,0.477,0.591,0.672,0.558,0.696,0.819,0.776,1.000,0.664,0.539,-0.045,-0.019,-0.245,-0.198
lLOC,0.106,0.489,0.605,0.647,0.551,0.690,0.772,0.707,0.664,1.000,0.517,-0.059,-0.046,-0.258,-0.144


Interpret correlations

Now, just show top correlations

In [7]:
# Define a function
hiCor <- function(x, level){
  res <- cor(x,method="spearman");
  res1 <- res; res1[res<0] <- -res[res < 0];
  for (i in 1:dim(x)[2]){
    res1[i,i] <- 0;
  }
  sel <- apply(res1,1,max) > level;
  res[sel,sel];
}
hiCor(data,.7)

,lNS,lNM,lNF,lNMR,lND,lLA,lLD,lLOC,lEXP,lREXP
lNS,1.000,0.755,0.650,0.652,0.577,0.512,0.477,0.489,-0.027,-0.019
lNM,0.7551,1.0000,0.8561,0.7432,0.7438,0.6532,0.5907,0.6050,-0.0092,0.0063
lNF,0.6501,0.8561,1.0000,0.7702,0.8615,0.7571,0.6716,0.6472,-0.0160,0.0031
lNMR,0.652,0.743,0.770,1.000,0.835,0.736,0.696,0.690,-0.059,-0.048
lND,0.577,0.744,0.862,0.835,1.000,0.868,0.819,0.772,-0.054,-0.031
lLA,0.512,0.653,0.757,0.736,0.868,1.000,0.776,0.707,-0.031,-0.018
lLD,0.477,0.591,0.672,0.696,0.819,0.776,1.000,0.664,-0.045,-0.019
lLOC,0.489,0.605,0.647,0.690,0.772,0.707,0.664,1.000,-0.059,-0.046
lEXP,-0.0272,-0.0092,-0.0160,-0.0593,-0.0536,-0.0314,-0.0447,-0.0591,1.0000,0.9595
lREXP,-0.0191,0.0063,0.0031,-0.0481,-0.0311,-0.0182,-0.0188,-0.0457,0.9595,1.0000


## So much correlation is typical in SE
* How to select an orthogonal subset of predictors?

Lets take a look at principal components

In [8]:
plot(1:15,cumsum(prcomp(vars, retx=F,scale=T)$sdev^2)/sum(prcomp(vars, retx=F,scale=T)$sdev^2),ylim=c(0,1),xlab="Number of coponents",ylab="Fraction of variance");
res<-prcomp(vars, retx=F,scale=T)$rotation[,1:5];
resAbs <- res;
resAbs[res<0] <- -res[res<0];
for (i in 1:5)
  print(t(res[resAbs[,i]>.3,i,drop=FALSE]));

     lNM  lNF lNLgn lNMR  lND  lLA lLD
PC1 0.32 0.33  0.31 0.34 0.34 0.31 0.3
     lEXP lREXP lSEXP
PC2 -0.65 -0.66 -0.33
    isBad FIX
PC3  0.84 0.5
      lNS lSEXP
PC4 -0.46  0.54
    isBad   FIX
PC5   0.5 -0.84


plot without title

* Regress each predictor on the remaining predictors
eliminate with the highest adjR^2

In [9]:
res <- c();
vnam <- names(data);
for (i in 2:dim(data)[2]){
  fmla <- as.formula(paste(vnam[i],paste(vnam[-c(1,i)],collapse="+"),sep="~"));
  res <- rbind(res,c(i,round(summary(lm(fmla,data=data))$r.squared,2)));
}
row.names(res) <- vnam[res[,1]];
res[order(-res[,2]),];

lEXP,12.00,0.95
lREXP,13.00,0.95
lND,7.00,0.94
lNF,4.00,0.91
lNM,3.00,0.87
lNMR,6.00,0.86
lLA,8.00,0.81
lNS,2.0,0.8
lLD,9.00,0.76
lNLgn,5.00,0.72
lLOC,10.00,0.65


### Finally select the model
  Note, that exploring only the predictor space is not leading to
     multiple comparisons issue!

  * replace lSEXP by lEXP: simpler 
  * why lLOC, lLA, lLD, lND, lNlgn: keep lND as most stable, keep lLOC?

In [11]:
fmla ~ isBad ~ lNS+lLA+FIX+lLOC+lINT+lEXP

fmla ~ isBad ~ lNS + lLA + FIX + lLOC + lINT + lEXP

* We'll need to do prediction, so drop last few years

In [12]:
data1 <- data;
data1$from <- risk$from/3600/24/365.25+1970;
dataFit <- data1[data1$from<=1997,]; #80%
dataTest <- data1[data1$from>1997,]; #20%

In [13]:
mod <- glm(isBad ~ lNS+lLA+FIX+lLOC+lINT+lEXP,family=binomial,data=dataFit);
summary(mod); 


Call:
glm(formula = isBad ~ lNS + lLA + FIX + lLOC + lINT + lEXP, family = binomial, 
    data = dataFit)

Deviance Residuals: 
   Min      1Q  Median      3Q     Max  
-0.842  -0.225  -0.169  -0.132   3.327  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -5.79427    0.45010  -12.87  < 2e-16 ***
lNS          0.35650    0.10268    3.47  0.00052 ***
lLA          0.17910    0.04751    3.77  0.00016 ***
FIX          0.53314    0.12943    4.12  3.8e-05 ***
lLOC         0.13590    0.04948    2.75  0.00603 ** 
lINT         0.01788    0.00925    1.93  0.05322 .  
lEXP        -0.10757    0.03829   -2.81  0.00497 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 3011.5  on 13480  degrees of freedom
Residual deviance: 2739.0  on 13474  degrees of freedom
AIC: 2753

Number of Fisher Scoring iterations: 7


Note AIC: 2753

threfore lINT probably not important

### Good to order by variance explained (Anova Deviance/Df)

In [14]:
anova(mod, test="Chi");

,Df,Deviance,Resid. Df,Resid. Dev,Pr(>Chi)
NULL,NA,NA,13480,3012,NA
lNS,1.0e+00,1.7e+02,1.3e+04,2.8e+03,5.3e-38
lLA,1.0e+00,6.5e+01,1.3e+04,2.8e+03,7.0e-16
FIX,1.0e+00,1.7e+01,1.3e+04,2.8e+03,3.0e-05
lLOC,1.0e+00,1.1e+01,1.3e+04,2.8e+03,1.1e-03
lINT,1.0e+00,5.5e+00,1.3e+04,2.7e+03,1.9e-02
lEXP,1.0e+00,7.7e+00,1.3e+04,2.7e+03,5.4e-03


A similar picture, lINT does not explain as much as others

In [15]:
mod <- glm(isBad ~ lNS+lND+FIX+lEXP+lINT,family=binomial,data=dataFit);
summary(mod);


Call:
glm(formula = isBad ~ lNS + lND + FIX + lEXP + lINT, family = binomial, 
    data = dataFit)

Deviance Residuals: 
   Min      1Q  Median      3Q     Max  
-0.851  -0.223  -0.170  -0.135   3.235  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -4.48210    0.28367  -15.80  < 2e-16 ***
lNS          0.31994    0.11021    2.90   0.0037 ** 
lND          0.36503    0.05354    6.82  9.2e-12 ***
FIX          0.52993    0.12873    4.12  3.8e-05 ***
lEXP        -0.10508    0.03799   -2.77   0.0057 ** 
lINT         0.01847    0.00941    1.96   0.0496 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 3011.5  on 13480  degrees of freedom
Residual deviance: 2750.4  on 13475  degrees of freedom
AIC: 2762

Number of Fisher Scoring iterations: 7


### Slightly higher AIC: 2762, but a simpler model
   * don't chase the best fit, as it leads to overfitting

In [16]:
anova(mod, test="Chi");#it is good to order predictors by deviance explained

,Df,Deviance,Resid. Df,Resid. Dev,Pr(>Chi)
NULL,NA,NA,13480,3012,NA
lNS,1.0e+00,1.7e+02,1.3e+04,2.8e+03,5.3e-38
lND,1.0e+00,6.4e+01,1.3e+04,2.8e+03,1.1e-15
FIX,1.0e+00,1.8e+01,1.3e+04,2.8e+03,2.6e-05
lEXP,1.0e+00,9.3e+00,1.3e+04,2.8e+03,2.4e-03
lINT,1.0e+00,3.8e+00,1.3e+04,2.8e+03,5.0e-02


* Note: use test="F" for linear models (lm)
* Note: R's anova order matters in variance explained, as
  each SS is based on the residuals from predictors going before it
* Alternative sum of squares for ANOVA are obtained via drop1
*  In this case it is based on the residuals of remaining predictors (not
  just of preceeding predictors

In [17]:
drop1(mod, test="Chi");

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,2750,2762,NA,NA
lNS,1.0e+00,2.8e+03,2.8e+03,8.6e+00,3.3e-03
lND,1.0e+00,2.8e+03,2.8e+03,4.4e+01,2.6e-11
FIX,1.0e+00,2.8e+03,2.8e+03,1.7e+01,4.1e-05
lEXP,1.0e+00,2.8e+03,2.8e+03,7.5e+00,6.1e-03
lINT,1.00,2754.22,2764.22,3.84,0.05


### Variance inflation factor
 http://en.wikipedia.org/wiki/Variance_inflation_factor

In [19]:
library(car)
vif(mod);

lNS  lND  FIX lEXP lINT 
 2.3  3.1  1.2  1.0  1.9

￼
* Is less than 5 (max is 3.1 for lND), but lower would be better

### Exercise: other diagnostics
* is model stable if some data is dropped?
* what if we change data as in GDF?
* is there a time trend 
    isBad ~ lNS+lND+FIX+lEXP+lINT+from
* independece (residuals)

Now interpret the model
* What do estimated coefficients mean?
* Increase NS from 1 to 2, but other predictors matter
* threfore pick values that are reasonable or a median

In [20]:
try <- dataFit[1:2,];
for (i in dim(dataFit)[2])
  try[,i] <- median(dataFit[,i]);
try[1,"lNS"] <- 0;
try[2,"lNS"] <- log(2);
res <- 1/(1+exp(-predict(mod,try)));
res[2]*(1-res[1])/res[1]/(1-res[2]);

2 
1.9

### The answer is how much the risk increases if number of subsystems touched is increased from 1 to 2
  * First ratio of risk then 
  * ratio of odds

# Do prediction

In [ ]:
predicted <- 1/(1+exp(-predict(mod,dataTest)));
tapply(predicted, dataTest$isBad, mean)

## Good to know
* predicted probability is almost three times higher for the MRs that break patches

### Traditional performance:

In [ ]:
for (cutof in c(.01, .015, .02, .03, .04, .045,.1)){
  res <- table(predicted>cutof, dataTest$isBad);
  type1 <- res[2,1]/(res[1,1]+res[2,1]);
  type2 <- res[1,2]/(res[1,2]+res[2,2]);
  recall <- 1 - type2;
  precision <- res[2,2]/(res[2,1]+res[2,2]);
  print (c(cutof,type1,type2,recall,precision));
}

### A single number such as ROC typically makes little sense:
* in this case the primary concern is decent recall

### Exercise:
* compare to a random predictor
* compare to a simple predictor, e.g, more than one subsystem

### Try other prediction methods

In [ ]:
library(rpart);

fmla <- isBad ~ lNS+lND+FIX+lINT+lEXP+lLA;
er <- c();
for (w in c(20,40,60,80, 100, 110)){ 
  cart.fit <- rpart(fmla,data=dataFit,method="class",weights=as.numeric(dataFit$isBad)*w+1);
  cart.pred <- predict(cart.fit,newdata=dataTest,type="class");
  res <- table(cart.pred, dataTest$isBad);
  type1 <- res[2,1]/(res[1,1]+res[2,1]);
  type2 <- res[1,2]/(res[1,2]+res[2,2]);
  recall <- 1 - type2;
  precision <- res[2,2]/(res[2,1]+res[2,2]);
  er <- rbind (er, c(w,type1,type2,recall,precision));
}

In [ ]:
### Slightly worse than the regression model